In [1]:
import time
import pandas as pd
import numpy as np
from GEECSPythonAPI import GEECSDevice
from GEECSPythonAPI import OptimizationControl
import yaml

In [2]:
# Axicon Horizontal vertical looking at minimizing STD 
# yamlfile="AxiconHorizontalVertical.yaml"
# evaluator_str = "-abs(np.std(outs))"
# TestPoints_dict = {"AxiconHorizontal":0.5,"AxiconVertical":0.5}
# InitialPoints_dict = {"AxiconHorizontal":[0.85,0.1],"AxiconVertical":[0.85,0.1]}
#tolerance=0.001

#greenie horizontal looking at lp mode 
yamlfile="PWfocusLpMode.yaml"
evaluator_str = "abs(np.median(outs)-900)"
TestPoints_dict = {"heatx":0.7970}
InitialPoints_dict = {"heatx":[0.7970,0.7930]}
tolerance=0.001

#hpd focal scan
# yamlfile="PWfocusLpMode.yaml"
# evaluator_str = "np.median(outs)"
# TestPoints_dict = {"exposure":0.0005}
# InitialPoints_dict = {"exposure":[0.00025,0.00007]}
# tolerance=0.01

# yamlfile="AxiconHorizontalVertical.yaml"
# evaluator_str = "-abs(np.std(outs))"
# TestPoints_dict = {"AxiconHorizontal":0.5,"AxiconVertical":0.5}
# InitialPoints_dict = {"AxiconHorizontal":[0.85,0.1],"AxiconVertical":[0.85,0.1]}
#tolerance=0.01


# this one is good for aligning to a crosshair
#    return {'f': -abs(np.median(outs)-672)}
#This is for minimizing fluctuation of the data
#    return {'f': eval(evaluator_str)}

In [3]:
#define a "yaml file" with no entries for variables. These will be filled in used the "bella_config" yamml file
YAML = """
xopt:
    dump_file: dump.yaml
generator:
    name: upper_confidence_bound
    n_initial: 5
    optim:
        num_restarts: 1
    acq:
        proximal_lengthscales: [.0001]

evaluator:
    function: __main__.geecs_measurement

vocs:
    variables:
        {}
    objectives: {f: "MINIMIZE"}

"""
yaml_output = yaml.safe_load(YAML)

with open(yamlfile, "r") as stream:
    try:
        bella_config=yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
yaml_output

{'xopt': {'dump_file': 'dump.yaml'},
 'generator': {'name': 'upper_confidence_bound',
  'n_initial': 5,
  'optim': {'num_restarts': 1},
  'acq': {'proximal_lengthscales': [0.0001]}},
 'evaluator': {'function': '__main__.geecs_measurement'},
 'vocs': {'variables': {}, 'objectives': {'f': 'MINIMIZE'}}}

In [4]:
optimization_controls=[]
ranges=[]
obj_tags=[]
normalize_ranges=False
cv_tags=dict(bella_config['control_variables'])
optimization_objects_dict=dict()
optimization_function_dict=dict()
for cv_tag, cv_config in cv_tags.items():
    if cv_config['set']:
        #find the variables in the config yaml file chosen as select and create a control object for them
        opt_contr=OptimizationControl(device_name=cv_config['device_name'],variable=cv_config['device_variable'],bounds=cv_config['range'])
        opt_contr.device_initialize()
        #optimization_controls.append(opt_contr)
        
        #create a dictionary with the keys that matech what are put into the VOCS structure that map to the 
        #xopt control objects so that they can be called later in geecs_measurement
        optimization_objects_dict[cv_tag]=opt_contr
        
        
        #calculate normalized ranges
        ranges.append(cv_config['range'])
        range_offset=[0,cv_config['range'][1]-cv_config['range'][0]]
        range_norm=[0,1]
        print(range_offset)
        
        #update the VOCS structure to use values taken from the config file
        if normalize_ranges:
            yaml_output['vocs']['variables'][cv_tag]=[0,1]
            yaml_output['generator']['acq']['proximal_lengthscales'].append(cv_config['proximal_lengthscale']/range_offset[1])
        else:
            yaml_output['vocs']['variables'][cv_tag]=cv_config['range']
    
    
    if cv_config['objective']:
        opt_contr=OptimizationControl(device_name=cv_config['device_name'],variable=cv_config['device_variable'])
        optimization_function_dict[cv_tag]=opt_contr
        opt_contr.device_initialize()

64194
[0, 0.0040000000000000036]
64196


In [5]:
outputs=optimization_function_dict

def geecs_measurement(input_dict):

    for i in list(input_dict.keys()):
        set_val=float(input_dict[i])
        print('set '+str(i)+' to '+str(set_val))
        optimization_objects_dict[i].xopt_set(float(set_val))
    time.sleep(1)
    outs=[]
    counter=0
    while counter<bella_config['optimization_parameters']['shots_per_step']:
        for i in outputs:
            val=float(optimization_function_dict[i].xopt_get())
            outs.append(val)
        counter=counter+1
    print(outs)

###############
## TO DO
###############
## Add some data processing on the resulting on the acquired data.
## for example, remove the extreme values, then maybe take a median
## also need to test with passing all acquired data points rather than some stat

# this one is good for aligning to a crosshair
#    return {'f': -abs(np.median(outs)-672)}
#This is for minimizing fluctuation of the data
#    return {'f': -abs(np.std(outs))}
    return {'f': eval(evaluator_str)}

In [6]:
geecs_measurement(TestPoints_dict)

set heatx to 0.797
can set
[356.316864, 392.190094, 399.082428, 386.523529, 391.63504, 380.133881, 266.300934, 376.181854, 383.188507, 390.989136, 391.957306, 393.989502, 387.3396, 389.000549, 385.192963]


{'f': 512.6604}

In [7]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt

X = Xopt(config=yaml_output)
X


            Xopt
________________________________
Version: 1.2.2
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: false, dump_file: dump.yaml, max_evaluations: null}
generator:
  name: upper_confidence_bound
  optim: {num_restarts: 1, raw_samples: 20, sequential: true, use_nearby_initial_points: true,
    max_travel_distances: null}
  acq:
    proximal_lengthscales: [0.0001]
    use_transformed_proximal_weights: true
    monte_carlo_samples: 128
    beta: 2.0
  model:
    function: xopt.generators.bayesian.models.standard.create_standard_model
    kwargs: {use_conservative_prior_lengthscale: false, use_conservative_prior_mean: false,
      use_low_noise_prior: false}
  n_initial: 5
evaluator:
  function: __main__.geecs_measurement
  max_workers: 1
  function_kwargs: {}
  vectorized: false
vocs:
  variables:
    heatx: [0.793, 0.797]
  constraints: {}
  objectives: {f: MINIMIZE}
  constants: {}
  linked_variables: {}


In [8]:
X.evaluate_data(pd.DataFrame(InitialPoints_dict))


set heatx to 0.797
can set
[394.616852, 439.227936, 385.370148, 382.100159, 386.458282, 374.518707, 364.823883, 277.365295, 368.261383, 384.690979, 381.831635, 385.835297, 381.584686, 375.869446, 387.555756]
set heatx to 0.793
can set
[1453.146484, 1375.33606, 1480.268433, 1430.846802, 1370.204956, 1367.744873, 1533.032227, 1379.944336, 1416.405884, 1385.797729, 1351.316528, 1347.680176, 1385.854126, 1370.629395, 1446.367187]


,heatx,f,xopt_runtime,xopt_error
1,0.797,517.899841,9.147543,False
2,0.793,485.797729,9.662985,False


In [9]:
import torch
import matplotlib.pyplot as plt
n_steps = 10

# test points for plotting
#test_x = torch.linspace(*X.vocs.bounds.flatten(),50).double()

for i in range(n_steps):
    print('step'+str(i))
    t0=time.monotonic()
    X.step()
    t1=time.monotonic()
    print('step'+str(i)+".1: "+str(t1-t0))
    # get the Gaussian process model from the generator
    model = X.generator.model
    t2=time.monotonic()
    print('step'+str(i)+".2: "+str(t2-t0))
    # get acquisition function from generator
    acq = X.generator.get_acquisition(model)
    t3=time.monotonic()
    print('step'+str(i)+".3: "+str(t3-t0))



step0
set heatx to 0.7953174802423376
can set
[705.327393, 705.998047, 704.223938, 707.035645, 697.946838, 702.760559, 703.867981, 703.777588, 696.739258, 702.042297, 703.152466, 707.010803, 715.127563, 711.288086, 709.486694]
step0.1: 10.203000000212342
step0.2: 10.203000000212342
step0.3: 10.219000000040978
step1
set heatx to 0.7959479920018708
can set
[602.573364, 550.995239, 530.931824, 634.673706, 675.259766, 664.641541, 698.173889, 687.823608, 685.68457, 681.502808, 696.914978, 696.68689, 597.221313, 651.01709, 539.942932]
step1.1: 9.984000000171363
step1.2: 9.984000000171363
step1.3: 9.984000000171363
step2
set heatx to 0.7969135452748264
can set
[390.558899, 387.865204, 385.714722, 386.570435, 370.233246, 390.572235, 391.089172, 385.919647, 389.944733, 404.711517, 387.468506, 387.5448, 388.619873, 385.681824, 389.853241]
step2.1: 10.015999999828637
step2.2: 10.015999999828637
step2.3: 10.015999999828637
step3
set heatx to 0.7969198799828688
can set
[389.77774, 400.350189, 390.4

In [10]:
X.data

,heatx,f,xopt_runtime,xopt_error
1,0.797000,517.899841,9.147543,False
2,0.793000,485.797729,9.662985,False
3,0.795317,195.776062,9.782435,False
4,0.795948,235.358459,9.570393,False
5,0.796914,512.134796,9.509259,False
6,0.796920,509.136932,9.251192,False
7,0.796910,278.688232,9.165457,False
8,0.795290,30.613342,9.634753,False
9,0.794993,126.533081,9.518692,False
10,0.796828,484.486145,9.766968,False


In [11]:
X.generator.generate(1)

,heatx
0,0.793909
